In [1]:
# 코랩에서 mecab 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2022-12-05 01:50:40--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-12-05 01:50:40--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7e26bf465b0b06e3143866734d.dl.dropboxusercontent.com/cd/0/get/ByD7QmlkeWK0T3z3TkNgklUDPbM8P_Rg8nYpC94ueK1LNYOdI9cF6f5QOVk9xHIhoIpCCHvyBdL7quFKFvSAXox31q-sNm2rWaJLbZ74gJr7lityKH6ohOnKYZ_Vz2pWLcbVQOj5z8z_nfDTnm-Ghi5I_0xU-t-BdFwLGMXk9lXqSiW84CKzbMu3Jo38BeTJ308/file?dl=1# [following]
--2022-12-05 01:50:41--  https://uc7e26bf465b0b06e3143866734d.dl.dropboxusercontent.com/cd/0/get/ByD7QmlkeWK0

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Mecab
from tqdm.notebook import tqdm

data = pd.read_csv('/content/drive/MyDrive/네이버_뉴스_추천/유사도/1.prep_data_dotsplits.csv')
data

,question,answer
0,정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다. 부디 자세한 설명해주시면 ...,차량의 경우 현재 점유자인 아버지를 피고로 하여 차량 인도 청구소송 및 차량 점유 ...
1,1년 전에 대포 통장이로 인해 피해를 봐서 재판 결과가 불기소 유통 지을 받았거든요...,소장 내용에 대한 검토가 필요하므로 소장을 지참하고 상담을 받으시기 바랍니다
2,연고도 다르고 일면식도 없는 5살위의 형을 일을 통해 알게 됐습니다.제가 주식투자를...,별도의 구체적 상담이 필요하나 실제 채무가 없음에도 공증을 가지고 채권자가 강제집행...
3,1 21615 이행권고결정판결 2 211523 채권압류 및 추심명령 인용 21153...,별도의 구체적 상담이 필요하나 이행 권고 결정은 기판력의 제한이 없으므로 소멸시효 ...
4,4천만 원 정도의 피해가 있고 고소를 했는데 인지사건으로 넘어가서 검찰에서 수사 중...,가해자가 위 사건으로 구속 기소되었다면 가해자를 피고로 하여 민사소송 및 강제집행으...
...,...,...
18160,제가 작년 8월쯤 승소 판결 받은 후에 압류 절차를 진행을 해야 하는데 혼자 스스로...,통장 압류 방법에 관하여 질문하셨습니다. 민사소송에서 승소를 하여 그 판결이 확정된...
18161,민사재판 패소 후 현재 등본은 부모님댁으로 되어있습니다. 나와서 따로 혼자 사는데 ...,민사소송 패소 후 부모님 집에 압류가 들어올 수 있는지와 관련하여 질문하셨습니다.민...
18162,추완항소이고 제가 피고인이예요 공시송달로 소송을 모르고 있다가 처음부터 끝까지 공시...,민사소송 항소이유를 잘못 기재한 것과 관련하여 질문하셨습니다. 부동산 임대차에 있어...
18163,작년 여름 사기 피해 금액 약 35만 원을 당하여 형사고소를 하였고 1월경 형사 고...,민사소송 소장에 소송비용 청구를 빠드린 경우와 관련하여 질 무하셨습니다. 민사소송에...


In [4]:
# 다음 뉴스 본문
df = pd.read_csv('/content/drive/MyDrive/네이버_뉴스_추천/유사도/daum_news.csv')
df = df.drop_duplicates('URL')
df = df.drop_duplicates('Content')
df = df.dropna(subset=['Content'])
df = df.reset_index(drop=True)
df

,Content,DomainID,MainCategory,PhotoURL,Press,Stickers,SubCategory,Title,URL,WritedAt,Writer
0,"기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 , ,“내가 조금만 다르게 행...",0,문화,https://img2.daumcdn.net/thumb/R658x0.q70/?fna...,중앙일보,"{'article_id': '20221107000430780', '좋아요': 0, ...",건강,살아가는 사람들의 고통 곁에 선다는 것 [건강한 가족],https://v.daum.net/v/20221107000430780,2022. 11. 7. 10:08,입력
1,"인터뷰 리타 다스 모더나 부사장,날이 추워지면서 코로나19 재확산 우려가 커지고 있...",0,문화,https://img1.daumcdn.net/thumb/R658x0.q70/?fna...,중앙일보,"{'article_id': '20221107000442786', '좋아요': 5, ...",건강,코로나 걸리고 백신 맞았어도 재감염 우려…‘2가 백신’ 접종해야 [건강한 가족],https://v.daum.net/v/20221107000442786,2022. 11. 7. 00:04,권선미
2,"[한국경제TV 엄수영 기자],아스트라제네카는 4일(현지시간) 영아의 호흡기 세포융합...",0,문화,https://img1.daumcdn.net/thumb/R658x0.q70/?fna...,한국경제TV,"{'article_id': '20221105000607068', '좋아요': 0, ...",건강,"아스트라제네카, 영유아 백신 니르세비맙 EU 승인",https://v.daum.net/v/20221105000607068,2022. 11. 5. 00:06,엄수영
3,일주일 전부터 수능일 일정에 맞춰 생활 조정해야2023년 수학능력시험이 2주 앞으로...,0,문화,https://img4.daumcdn.net/thumb/R658x0.q70/?fna...,코메디닷컴,"{'article_id': '20221105170039784', '좋아요': 0, ...",건강,"코 앞으로 다가온 수능, ‘컨디션’ 관리는?",https://v.daum.net/v/20221105170039784,2022. 11. 5. 17:00,김혜원
4,"[오늘의 건강] 제철 맞은 대하, 가리비, 해삼의 건강 효능중부와 경상권 아침기온이...",0,문화,https://img2.daumcdn.net/thumb/R658x0.q70/?fna...,코메디닷컴,"{'article_id': '20221106060014096', '좋아요': 4, ...",건강,제철 해산물로 면역력 ‘쑥’ 키워볼까,https://v.daum.net/v/20221106060014096,2022. 11. 6. 06:00,이지원
...,...,...,...,...,...,...,...,...,...,...,...
25664,[이뉴스투데이 이지혜 기자] LG생활건강이 2023년 정기 임원인사 첫 여성 대표이...,0,문화,https://img2.daumcdn.net/thumb/R658x0.q70/?fna...,마이데일리,"{'article_id': '20221124175952326', '좋아요': 0, ...",생활정보,"LG생활건강, 공채출신 이정애 대표 첫 여성 사장 발탁",https://v.daum.net/v/20221124175952326,NaN,입력
25665,[이뉴스투데이 이지혜 기자] 아시아나항공이 더현대 서울에 ‘여행세포 연구소’ 팝업...,0,문화,https://img4.daumcdn.net/thumb/R658x0.q70/?fna...,마이데일리,"{'article_id': '20221124180015377', '좋아요': 0, ...",생활정보,"아시아나항공, 미주·유럽 항공권 15% 할인…더현대 서울에 체험형 팝업스토어 오픈",https://v.daum.net/v/20221124180015377,NaN,입력
25666,"이정애 LG생건 사장·박애리 지투알 부사장 등 2명 CEO 선임,‘최초’의 대명사 ...",0,문화,https://img4.daumcdn.net/thumb/R658x0.q70/?fna...,이데일리,"{'article_id': '20221124180219469', '좋아요': 0, ...",생활정보,"LG그룹, 4대 그룹 최초 여성 CEO 선임",https://v.daum.net/v/20221124180219469,2022. 11. 24. 18:02,윤정훈
25667,"배달의 민족 주문·결제 오류에 고객 불편,쿠팡이츠 “치킨 주문 원활하지 않다” 양해...",0,문화,https://img2.daumcdn.net/thumb/R658x0.q70/?fna...,이데일리,"{'article_id': '20221124215748248', '좋아요': 6, ...",생활정보,월드컵에 치킨 주문 ‘폭발’…배민 서버 오류 ‘대혼란’,https://v.daum.net/v/20221124215748248,2022. 11. 24. 21:57,윤정훈


In [5]:
def preprocessing(idx):
    content = df.loc[idx, 'Content']
    # 숫자 한글 영문 , . 공백을 제외한 문자 제거
    content_pre = re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣. ]', '', content)
    # 소수 및 정수 제거
    content_pre = re.sub('\d+.\d*', '',content_pre)
    # url 및 이메일 제거
    content_pre = re.sub("([a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)", '', content_pre)
    # 다수의 점 (ex : ...) 점 한개로 대체
    content_pre = re.sub("\.+\.", '.', content_pre)
    # 다수의 공백 축소
    content_pre = re.sub(' +', ' ', content_pre)

    # 기자명, 언론사명 stopword 처리
    wrt = df.loc[idx, 'Writer']
    press = df.loc[idx, 'Press']

    # stopword 추가해서 처리
    stop_word = [f'{wrt}', f'{press}', '기자', 'Copyrights',  '무단 전재 및 재배포 금지']
    content_pre2 = content_pre
    for stp in stop_word :
        content_pre2 = content_pre2.replace(stp, '')
    return content_pre2

In [6]:
import re
df_content = pd.DataFrame(columns=['content'])
for idx in tqdm(df.index):
    temp_content = preprocessing(idx)
    temp_df = pd.DataFrame({'content' : temp_content}, index=[0])
    df_content = pd.concat([df_content, temp_df])
df_content = df_content.reset_index(drop=True)
df_content

  0%|          | 0/25669 [00:00<?, ?it/s]

,content
0,기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 내가 조금만 다르게 행동했더라...
1,인터뷰 리타 다스 모더나 부사장날이 추워지면서 코로나재확산 우려가 커지고 있다. 새...
2,아스트라제네카는 현지시간 영아의 호흡기 세포융합 바이러스RSV를 예방하는 치료제...
3,일주일 전부터 수능일 일정에 맞춰 생활 조정해야 수학능력시험이 앞으로 다가왔다. 큰...
4,오늘의 건강 제철 맞은 대하 가리비 해삼의 건강 효능중부와 경상권 아침기온이 이하로...
...,...
25664,이뉴스투데이 이지혜 LG생활건강이 정기 임원인사 첫 여성 대표이사로 공채 출신 이...
25665,이뉴스투데이 이지혜 아시아나항공이 더현대 서울에 여행세포 연구소 팝업스토어를 연...
25666,이정애 LG생건 사장박애리 지투알 부사장 등 CEO 선임최초의 대명사 이정애 사장 ...
25667,배달의 민족 주문결제 오류에 고객 불편쿠팡이츠 치킨 주문 원활하지 않다 양해 공지 ...


In [7]:
def get_df_token(data):

    mecab = Mecab()
    q_pos = []

    for i in tqdm(range(len(data))):
        q_pos.append(mecab.pos(data['content'][i]))
    data['content_token'] = q_pos

    pos = ["NNG","NNP",   # 명사
         "VV","VA"   # 동사
         ]

    data['content_token'] = data['content_token'].apply(lambda x: [t for (t, p) in x if p in pos])

    return data

def get_ques_token(content_token):
    mecab = Mecab()
    ques_token = mecab.pos(content_token)
    ques_token = ques_remove(ques_token)

    return ques_token

def ques_remove(content_token) :

    pos = ["NNG","NNP",   # 명사
           "VV","VA"   # 동사
           ]

    ques_token = [[t for (t, p) in content_token if p in pos]]

    return ques_token

def jaccard_similarity(data, content_token):

    union = set(data).union(set(content_token))
    intersection = set(data).intersection(set(content_token))

    jaccard_sim = len(intersection) / len(union)  

    return jaccard_sim

def jaccard_high(data, content_token, num):
    
    # data: 데이터프레임, content_token: 입력한 텍스트(content_token), num: 자카드 유사도 상위 갯수
    data['jaccard_similarity'] = data['content_token'].apply(lambda x: jaccard_similarity(x, content_token[0]))

    return data[['content', 'content_token', 'jaccard_similarity']].sort_values(['jaccard_similarity'], ascending=False)[:num]

def tokenized_output(tokens):
    return tokens

def cos_similarity(data, content_token):

    tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                                       tokenizer=tokenized_output,
                                       preprocessor=tokenized_output,
                                       token_pattern=None)
    
    tfidf_data = tfidf_vectorizer.fit_transform(data['content_token'])
    tfidf_content_token = tfidf_vectorizer.transform(content_token)

    data['cosine_similarity'] = cosine_similarity(tfidf_data, tfidf_content_token).reshape(-1, )

    return data[['content', 'content_token', 'jaccard_similarity', 'cosine_similarity']].sort_values(['cosine_similarity'], ascending=False)

def cos_high(data):
    if data['cosine_similarity'][0] >= 0.6:
        return data['content'][0]

    else:
        return print('모델구현')


In [8]:
mecab = Mecab()
q_pos = []

for i in tqdm(range(len(df_content))):
    q_pos.append(mecab.pos(df_content['content'][i]))
df_content['content_token'] = q_pos

  0%|          | 0/25669 [00:00<?, ?it/s]

In [9]:
df_content['content_token']

0        [(기, VV), (고, EC), (허, IC), (휴정, NNG), (가톨릭, N...
1        [(인터뷰, NNG), (리타, NNG), (다스, NNG), (모더, NNP), ...
2        [(아스트라, NNP), (제, NP), (네, XSN), (카, NNG), (는,...
3        [(일, NR), (주일, NNBC), (전, NNG), (부터, JX), (수능,...
4        [(오늘, NNG), (의, JKG), (건강, NNG), (제철, NNG), (맞...
                               ...                        
25664    [(이, MM), (뉴스투데이, NNP), (이지혜, NNP), (LG, SL), ...
25665    [(이, MM), (뉴스투데이, NNP), (이지혜, NNP), (아시아나항공, N...
25666    [(이정애, NNP), (LG, SL), (생, XSN), (건, NNB+JX), ...
25667    [(배달, NNG), (의, JKG), (민족, NNG), (주문, NNG), (결...
25668    [(이원율, NNP), (대한민국, NNP), (축구, NNG), (대표, NNG)...
Name: content_token, Length: 25669, dtype: object

In [10]:
content1 = df_content.iloc[0, 0]
content1

'기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 내가 조금만 다르게 행동했더라면 그 친구가 죽지 않았을까요 한 여성은 여행 중에 교통사고로 친한 친구를 잃었다. 사고 이후 여성의 머릿속은 마치 고장 난 라디오처럼 그날 하지 말았어야 했던 행동과 해야 했던 행동을 떠올리고 또 떠올렸다. 사고가 난 지 이 지났지만 그는 생존자의 죄책감Survivors guilt으로 고통받고 있다. 지난달 새벽 이태원 참사에 대한 속보를 보며 나는 진료실에서 만났던 그를 떠올렸다. 소중한 친구를 잃어버린 사람 정신없이 몇 시간 동안 심폐소생술CPR을 하고도 죽음을 허망하게 목도할 수밖에 없었던 구조대원과 시민들 그저 멍한 채로 얼어붙어 꼼짝달싹하지 못한 채 그 광경을 지켜볼 수밖에 없었던 사람들이 떠올랐다. 어쩌면 그들도 그처럼 그날 하지 말았어야 했던 행동과 해야 했던 행동들을 수없이 떠올리며 괴로워하고 있을지 모른다. 생존자의 죄책감은 정신 질환 진단 및 통계편람The diagnostic and Statistical Manual of Mental Disorders에서도 트라우마 이후에 나타날 수 있는 주요한 증상 중 하나로 꼽고 있다. 어떤 이들은 많은 사람이 죽었음에도 자신이 살아남은 것 자체에 죄책감을 느끼고 누군가를 구하지 못한 것을 지나치게 자책하기도 한다. 이러한 죄책감은 트라우마로부터의 회복에 큰 방해가 되기도 하고 때로 자살과 같은 끔찍한 결과를 초래한다. 이들을 대상으로 한 지속적인 심리 지원과 돌봄이 매우 중요한 이유다. 생존자의 죄책감으로 괴로운 사람들에게 누군가 무심코 던진 한마디는 평생을 괴롭히기도 한다. 왜 하필 거기에 갔나 철없이 놀다 죽은 것을 애도해야 하나 스스로 선택해 간 것인데라는 등의 댓글은 벼랑 끝에 있는 생존자를 더 큰 고통의 나락으로 내몬다. 누구나 살다 보면 결과가 좋지 않았던 선택을 할 수 있고 삶과 죽음이 내 능력 밖의 일임을 절감하는 상황을 마주하곤 한다. 그러기에 우리는 예기치 않은 죽음 앞에 더 신중해지고 겸허해질 필요가

In [11]:
df_content

,content,content_token
0,기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 내가 조금만 다르게 행동했더라...,"[(기, VV), (고, EC), (허, IC), (휴정, NNG), (가톨릭, N..."
1,인터뷰 리타 다스 모더나 부사장날이 추워지면서 코로나재확산 우려가 커지고 있다. 새...,"[(인터뷰, NNG), (리타, NNG), (다스, NNG), (모더, NNP), ..."
2,아스트라제네카는 현지시간 영아의 호흡기 세포융합 바이러스RSV를 예방하는 치료제...,"[(아스트라, NNP), (제, NP), (네, XSN), (카, NNG), (는,..."
3,일주일 전부터 수능일 일정에 맞춰 생활 조정해야 수학능력시험이 앞으로 다가왔다. 큰...,"[(일, NR), (주일, NNBC), (전, NNG), (부터, JX), (수능,..."
4,오늘의 건강 제철 맞은 대하 가리비 해삼의 건강 효능중부와 경상권 아침기온이 이하로...,"[(오늘, NNG), (의, JKG), (건강, NNG), (제철, NNG), (맞..."
...,...,...
25664,이뉴스투데이 이지혜 LG생활건강이 정기 임원인사 첫 여성 대표이사로 공채 출신 이...,"[(이, MM), (뉴스투데이, NNP), (이지혜, NNP), (LG, SL), ..."
25665,이뉴스투데이 이지혜 아시아나항공이 더현대 서울에 여행세포 연구소 팝업스토어를 연...,"[(이, MM), (뉴스투데이, NNP), (이지혜, NNP), (아시아나항공, N..."
25666,이정애 LG생건 사장박애리 지투알 부사장 등 CEO 선임최초의 대명사 이정애 사장 ...,"[(이정애, NNP), (LG, SL), (생, XSN), (건, NNB+JX), ..."
25667,배달의 민족 주문결제 오류에 고객 불편쿠팡이츠 치킨 주문 원활하지 않다 양해 공지 ...,"[(배달, NNG), (의, JKG), (민족, NNG), (주문, NNG), (결..."


In [12]:
df_content = get_df_token(df_content)
df_content

  0%|          | 0/25669 [00:00<?, ?it/s]

,content,content_token
0,기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 내가 조금만 다르게 행동했더라...,"[기, 휴정, 가톨릭, 인천, 성모병원, 정신, 건강, 의학, 교수, 다르, 행동,..."
1,인터뷰 리타 다스 모더나 부사장날이 추워지면서 코로나재확산 우려가 커지고 있다. 새...,"[인터뷰, 리타, 다스, 모더, 부사장, 날, 코로나, 확산, 우려, 커지, 감염병..."
2,아스트라제네카는 현지시간 영아의 호흡기 세포융합 바이러스RSV를 예방하는 치료제...,"[아스트라, 카, 현지, 시간, 영아, 호흡기, 세포, 융합, 바이러스, 예방, 치..."
3,일주일 전부터 수능일 일정에 맞춰 생활 조정해야 수학능력시험이 앞으로 다가왔다. 큰...,"[전, 수능, 일정, 생활, 조정, 수학, 능력, 시험, 앞, 시험, 앞두, 불안감..."
4,오늘의 건강 제철 맞은 대하 가리비 해삼의 건강 효능중부와 경상권 아침기온이 이하로...,"[오늘, 건강, 제철, 맞, 대하, 가리비, 해삼, 건강, 효능, 중부, 경상, 아..."
...,...,...
25664,이뉴스투데이 이지혜 LG생활건강이 정기 임원인사 첫 여성 대표이사로 공채 출신 이...,"[뉴스투데이, 이지혜, 생활, 건강, 정기, 임원, 인사, 여성, 대표, 이사, 공..."
25665,이뉴스투데이 이지혜 아시아나항공이 더현대 서울에 여행세포 연구소 팝업스토어를 연...,"[뉴스투데이, 이지혜, 아시아나항공, 현대, 서울, 여행, 세포, 연구소, 팝업, ..."
25666,이정애 LG생건 사장박애리 지투알 부사장 등 CEO 선임최초의 대명사 이정애 사장 ...,"[이정애, 사장, 박애리, 지투알, 부사장, 선임, 최초, 대명사, 이정애, 사장,..."
25667,배달의 민족 주문결제 오류에 고객 불편쿠팡이츠 치킨 주문 원활하지 않다 양해 공지 ...,"[배달, 민족, 주문, 결제, 오류, 고객, 불편, 쿠팡, 치킨, 주문, 원활, 양..."


In [13]:
ques_token = get_ques_token(content1)
ques_token

[['기',
  '휴정',
  '가톨릭',
  '인천',
  '성모병원',
  '정신',
  '건강',
  '의학',
  '교수',
  '다르',
  '행동',
  '친구',
  '죽',
  '여성',
  '여행',
  '교통사고',
  '친구',
  '잃',
  '사고',
  '이후',
  '여성',
  '머릿속',
  '고장',
  '라디오',
  '그날',
  '하',
  '행동',
  '행동',
  '떠올리',
  '사고',
  '생존자',
  '죄책감',
  '고통',
  '받',
  '지난달',
  '새벽',
  '이태원',
  '참사',
  '속보',
  '보',
  '진료실',
  '친구',
  '사람',
  '시간',
  '동안',
  '심폐',
  '소생',
  '술',
  '하',
  '죽음',
  '목도',
  '없',
  '구조',
  '대원',
  '시민',
  '얼어붙',
  '하',
  '광경',
  '없',
  '사람',
  '그날',
  '하',
  '행동',
  '행동',
  '떠올리',
  '괴로워하',
  '생존자',
  '죄책감',
  '정신',
  '질환',
  '진단',
  '통계',
  '편람',
  '트라우마',
  '이후',
  '있',
  '증상',
  '꼽',
  '많',
  '사람',
  '죽',
  '자신',
  '살아남',
  '자체',
  '죄책감',
  '느끼',
  '구하',
  '지나치',
  '자책',
  '죄책감',
  '트라우마',
  '회복',
  '방해',
  '되',
  '하',
  '자살',
  '같',
  '결과',
  '초래',
  '대상',
  '지속',
  '심리',
  '지원',
  '중요',
  '이유',
  '생존자',
  '죄책감',
  '사람',
  '한마디',
  '평생',
  '괴롭히',
  '놀',
  '죽',
  '애',
  '선택',
  '댓글',
  '벼랑',
  '끝',
  '있',
  '생존자',
  '고통',
  '나락',
  '살',
  '결과',
 

In [14]:
result = jaccard_high(df_content, ques_token, 100)
result = cos_similarity(result, ques_token)

In [15]:
result

,content,content_token,jaccard_similarity,cosine_similarity
0,기고 허휴정 가톨릭대 인천성모병원 정신건강의학과 교수 내가 조금만 다르게 행동했더라...,"[기, 휴정, 가톨릭, 인천, 성모병원, 정신, 건강, 의학, 교수, 다르, 행동,...",1.000000,1.000000
11872,허심양 임상심리전문가 인터뷰온 나라가 슬픔에 잠겼다. 서울 한복판에서 가까운 사상자...,"[허심, 양, 임상, 심리, 전문가, 인터뷰, 나라, 슬픔, 서울, 복판, 사상자,...",0.139205,0.289618
1918,이태원 참사 후 트라우마 상담 기록 올린 생존자 인터뷰재난 이겨내는 유일한 방법은 ...,"[이태원, 참사, 후, 트라우마, 상담, 기록, 생존자, 인터뷰, 재난, 유일, 방...",0.128743,0.225944
210,스트레스를 넘어 안전생명에 위협될 만한 사건 겪으면 발생평생 겪을 확률 이상가까운 ...,"[스트레스, 넘, 안전, 생명, 위협, 사건, 겪, 발생, 평생, 겪, 확률, 이상...",0.137931,0.222380
1294,일반 스트레스와는 달라.주변의 정서적 지지 중요사고나 자연재해가 발생하면 몸만 다치...,"[일반, 스트레스, 주변, 정서, 지지, 중요, 사고, 자연재해, 발생, 몸, 다치...",0.136531,0.218702
...,...,...,...,...
24690,본인은 어느 정도의 청력을 가지고 있다고 생각하는가 필자가 청력 검사를 받은 사람들...,"[본인, 정도, 청력, 가지, 생각, 필자, 청력, 검사, 받, 사람, 묻, 말, ...",0.110656,0.043319
24514,당뇨병 환자라고 하면 이상 중년이 많이 걸린다고 여기지만 어린이 환자도 심심치 않게...,"[당뇨병, 환자, 하, 이상, 중년, 여기, 어린이, 환자, 있, 당뇨병, 김성언,...",0.119403,0.041821
1682,가천대 길병원 고기동 교수 입원 시 금연치료 병행 프로그램 제안담배가 몸에 좋지 않...,"[가천, 길, 병원, 고기동, 교수, 입원, 금연, 치료, 병행, 프로그램, 제안,...",0.104317,0.041516
112,심장신경성 실신은 확률화장실에서 일을 보다가 의식을 잃은 적 있는가 잠깐 동안의 실...,"[심장, 신경, 실신, 확률, 화장실, 일, 보, 의식, 잃, 있, 동안, 실신, ...",0.104803,0.041286
